# Object Detection API Demo

<table align="left"><td>
  <a target="_blank"  href="https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

Welcome to the [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection). This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image.

> **Important**: This tutorial is to help you through the first step towards using [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection) to build models. If you just just need an off the shelf model that does the job, see the [TFHub object detection example](https://colab.sandbox.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb).

# Setup

Important: If you're running on a local machine, be sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md). This notebook includes only what's necessary to run in Colab.

### Install

In [2]:
# !pip install -U --pre tensorflow=="2.*"
# !pip install --user Cython
# !pip install --user contextlib2
# !pip install --user pillow
# !pip install --user lxml
# !pip install --user jupyter
# !pip install --user matplotlib

Make sure you have `pycocotools` installed

In [3]:
# !pip install pycocotools

Get `tensorflow/models` or `cd` to parent directory of the repository.

In [1]:
import os
import pathlib


if "4thYearProject-master" in pathlib.Path.cwd().parts:
  while "4thYearProject-master" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('4thYearProject-master').exists():
  !git clone --depth 1 https://github.com/candiceliann/4thYearProject

Compile protobufs and install the object_detection package

In [2]:
os.getcwd()
os.chdir('/Users/AndrewBarrett/Downloads/4thYearProject-master/TensorflowObjectDetection/research')
os.getcwd()

'/Users/AndrewBarrett/Downloads/4thYearProject-master/TensorflowObjectDetection/research'

In [5]:
#pip install .

Processing /Users/AndrewBarrett/Downloads/4thYearProject-master/TensorflowObjectDetection/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1018409 sha256=1f384806e4ddb8b3851c8f8789d967d075bc4187c1ac21638d4dea7dcd3527f3
  Stored in directory: /private/var/folders/td/sd22gzd915d58j7tcdq_54h80000gn/T/pip-ephem-wheel-cache-u37jxlo0/wheels/72/61/d7/a9aadd14d1dbb7891682281391ab820ca54b1b9405df9af216
Successfully built object-detection
  Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1
Note: you may need to restart the kernel to use updated packages.


### Imports

Import the object detection module.

In [6]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

In [7]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [8]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [9]:
#MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_NAME = "mask_rcnn_inception_v2_coco_2018_01_28"
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = '/Users/AndrewBarrett/Downloads/4thYearProject-master/TensorflowObjectDetection/research/object_detection/data/mscoco_label_map.pbtxt'

NUM_CLASSES = 10

In [10]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

In [11]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = utils_ops.tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [12]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print(category_index)

{1: {'id': 1, 'name': 'person'}, 2: {'id': 2, 'name': 'bicycle'}, 3: {'id': 3, 'name': 'car'}, 4: {'id': 4, 'name': 'motorcycle'}, 5: {'id': 5, 'name': 'airplane'}, 6: {'id': 6, 'name': 'bus'}, 7: {'id': 7, 'name': 'train'}, 8: {'id': 8, 'name': 'truck'}, 9: {'id': 9, 'name': 'boat'}, 10: {'id': 10, 'name': 'traffic light'}}


In [13]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [14]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/Users/AndrewBarrett/Downloads/4thYearProject-master/LaneDetection/output_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.mp4")))

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)
TEST_IMAGE_PATHS

[PosixPath('/Users/AndrewBarrett/Downloads/4thYearProject-master/LaneDetection/output_images/Dashcam footage - M56 dangerous driving.mp4')]

In [15]:
out = cv2.VideoWriter('/Users/AndrewBarrett/Downloads/4thYearProject-master/LaneDetection/output_images/output_video.avi', cv2.VideoWriter_fourcc(
        'M', 'J', 'P', 'G'), 10, (1280, 720))

In [ ]:
with detection_graph.as_default():
    with utils_ops.tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object
        # was detected.
        detection_boxes = detection_graph.get_tensor_by_name(
            'detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class
        # label.
        detection_scores = detection_graph.get_tensor_by_name(
            'detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name(
            'detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name(
            'num_detections:0')
        cap = cv2.VideoCapture('/Users/AndrewBarrett/Downloads/4thYearProject-master/LaneDetection/output_images/Dashcam footage - M56 dangerous driving.mp4')

        while(cap.isOpened()):
            # Read the frame
            ret, frame = cap.read()

            # Recolor the frame. By default, OpenCV uses BGR color space.
            # This short blog post explains this better:
            # https://www.learnopencv.com/why-does-opencv-use-bgr-color-format/
            color_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            image_np_expanded = np.expand_dims(color_frame, axis=0)

            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores,
                    detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})

            # Visualization of the results of a detection.
            # note: perform the detections using a higher threshold
            vis_util.visualize_boxes_and_labels_on_image_array(
                color_frame,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8,
                min_score_thresh=.20)

            cv2.imshow('frame', color_frame)
            output_rgb = cv2.cvtColor(color_frame, cv2.COLOR_RGB2BGR)
            out.write(output_rgb)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        out.release()
        cap.release()
        cv2.destroyAllWindows()